In [10]:
from utils import preprocess_tweet
import gensim
import pandas as pd

C:\Users\jigna\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# **Data**

In [2]:
data = pd.read_csv("data/seven_days_tweets.csv")[["date", "tweet"]]
print(data.sample(5))

print("--------------------------------------------------------------------------")
print("Cantidad de tweets: ", len(data))
print("--------------------------------------------------------------------------")


             date                                              tweet
67973  2021-01-04                              @sherlocka007 🤣🤣🤣🤣🤣🤣🤣
68341  2021-01-04             @MConsuelofg La famosa 13 de Dr. House
65706  2021-01-05                       Pucha que desperté contento.
14099  2021-01-09                 @vale_ortega @CristianNeira Jajaja
31568  2021-01-07  Me cuesta pensar en una institución más sórdid...
--------------------------------------------------------------------------
Cantidad de tweets:  81612
--------------------------------------------------------------------------


# **Preprocessing**

In [14]:
# Se pre-procesan los tweets: Esto transforma cada
# tweet en una colección de palabras.
# processed_tweets corresponde a una lista de listas de palabras.

processed_tweets = data['tweet'].map(preprocess_tweet)
processed_tweets.head()

# Se van a eliminar tweets pequeños: con menos de 5 palabras después
# del preprocessing.
dropers = []
for ind, tweet in enumerate(processed_tweets):
    if len(tweet) < 5:
        dropers.append(ind) 

processed_tweets = processed_tweets.drop(dropers)
processed_tweets.head(5)

0     [jaja, papito, jugamos, wanderers, dijo:, matí...
1     [“pucha, usada”, fotos, bicicleta, lucas, 🤦🏽‍♀...
8     [recuerda, julio, lopez, blanco, agente, civil...
11    [ganas, contrato, años, precio, baja, cierto, ...
12           [mala, calidad, desenfocado, niun, brillo]
Name: tweet, dtype: object

In [19]:
# Se crea el vocabulario. 
dictionary = gensim.corpora.Dictionary(processed_tweets)
# Se quitan palabras que aparecen en menos de 20 tweets y las
# que aparecen en más del 50% del total de tweets (?).
dictionary.filter_extremes(no_below=20, no_above=0.5)
c = 0
for k, v in dictionary.iteritems():
    print(k,v)
    c += 1
    if c > 5: break
        
print("------------------------------------------")
print('Largo del diccionario: ', len(dictionary))
print("------------------------------------------")


0 jaja
1 matías
2 wanderers
3 acaso
4 bicicleta
5 fotos
------------------------------------------
Largo del diccionario:  3282
------------------------------------------


In [16]:
# Se transforman las palabras a vectores con el dictionary.
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]
print('tweet 0: ', bow_corpus[0])
print('tweet 1: ', bow_corpus[1])

tweet 0:  [(0, 1), (1, 1), (2, 1)]
tweet 1:  [(3, 1), (4, 2), (5, 1), (6, 1), (7, 1)]


In [17]:
# Se crea y usa el lda model
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary#.id2token

lda_model = LdaModel(
    corpus=bow_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state=1
)

In [18]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.062*"chile" + 0.052*"avenida" + 0.047*"santiago" + 0.038*"sentir" + 0.038*"voluntad" + 0.028*"fuego" + 0.024*"gracias" + 0.024*"foto" + 0.022*"acaba" + 0.021*"publicar"
Topic: 1 
Words: 0.028*"accidente" + 0.018*"gran" + 0.016*"basura" + 0.013*"vamos" + 0.012*"partido" + 0.011*"vehicular1" + 0.010*"jose" + 0.009*"general" + 0.009*"miguel" + 0.009*"pueblo"
Topic: 2 
Words: 0.031*"forma" + 0.030*"jadue" + 0.030*"delincuentes" + 0.029*"guerra" + 0.028*"imagino" + 0.028*"profesional" + 0.027*"premio" + 0.027*"renuncia" + 0.020*"bien" + 0.015*"personas"
Topic: 3 
Words: 0.021*"años" + 0.019*"vida" + 0.016*"amor" + 0.016*"educación" + 0.015*"demás" + 0.015*"seguro" + 0.015*"macul" + 0.015*"mala" + 0.015*"vivir" + 0.014*"niña"
Topic: 4 
Words: 0.035*"días" + 0.033*"bueno" + 0.031*"buena" + 0.029*"falta" + 0.028*"dijo" + 0.028*"hablar" + 0.027*"casi" + 0.027*"pasó" + 0.026*"muerte" + 0.026*"público"
